In [2]:
from text_denoiser import TextDenoiser
import torch
import torch.nn as nn
import torch.nn.functional as F

c:\Users\willd\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
denoiser = TextDenoiser(embed_dim=228)
# denoiser.load_state_dict(torch.load("../logs/train_test/models/saved_model.pt", map_location=device))
print(denoiser.decoder.weight.shape)
print(denoiser.embedder.weight.shape)
# denoiser.decoder.weight = nn.Parameter(denoiser.embedder.weight)

torch.Size([10053, 228])
torch.Size([10053, 228])


In [5]:
n_T = 40
data = torch.ones(32, 4, 228)
factor = torch.linspace(0, n_T-1, n_T)
ts = torch.randint(0, n_T, (data.shape[1],))
print(factor)
print(ts)

data = data * factor[None, ts, None]
print(data[:, 1, :].mean())



tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
        28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.])
tensor([23,  3, 38, 32])
tensor(3.)


In [121]:
texts = denoiser.vocab.lookup_tokens(torch.randint(0, len(denoiser.vocab), (5,)).tolist())
print(texts)
indices = torch.LongTensor(denoiser.vocab(texts)).to(device)

embeds = denoiser.embedder(indices)
embeds.shape

decoded = denoiser.decoder(embeds)
print(decoded.shape)
new_indices = torch.argmax(decoded, dim=-1)
print(new_indices)
print(denoiser.vocab.lookup_tokens(new_indices.tolist()))

print(denoiser.vocab(texts))


['mitchells', 'tegetthoff', 'wife', 'half', 'exceeding']
torch.Size([5, 10053])
tensor([4234, 3537,  836,  353, 7695])
['mitchells', 'tegetthoff', 'wife', 'half', 'exceeding']
[4234, 3537, 836, 353, 7695]


In [19]:
denoiser.sample(device, 1, 10)

['s <unk> plan sketches spikes managed tiger 1800 their acquire']

In [36]:
from transformers import BertConfig, BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [97]:
# Setup bert pipeline

def get_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    return outputs[0]


# text = "shovel bucket"
# tokens = tokenizer.tokenize(text)
# tokens = ["[CLS]"] + tokens + ["[SEP]"]

# token_ids = tokenizer.convert_tokens_to_ids(tokens)
# token_ids = torch.LongTensor(token_ids).unsqueeze(0)

# with torch.no_grad():
#     outputs = model(token_ids)
#     last_hidden_states = outputs[0]

# print(last_hidden_states[0, 1, 0:3])

emb1 = get_embedding("", model, tokenizer)
emb2 = get_embedding("bathroom", model, tokenizer)

print(emb1[0, 1].norm())
print(emb1.shape[-1] ** 0.5)

# Cosine similarity between "shovel" and "bucket" embeddings
# print(torch.nn.functional.cosine_similarity(last_hidden_states[0, 1:2], last_hidden_states[0, 2:3]))
# print(model.get_output_embeddings())

# Cosine similarity between embeddings
# print(torch.nn.functional.cosine_similarity(emb1[0, 0:1], emb2[0, 0]))

# inverse process: get text from embedding
tokens = tokenizer("Hello I'm the new king")["input_ids"]
print(tokens)
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(tokens))



tensor(15.0457, grad_fn=<NormBackward1>)
27.712812921102035
[101, 7592, 1045, 1005, 1049, 1996, 2047, 2332, 102]


"[CLS] hello i ' m the new king [SEP]"